In [7]:
import metpy.calc as mpcalc
from os import listdir
import numpy as np
from datetime import datetime as dt
import pandas as pd
import xarray as xr
import holoviews as hv
import panel as pn
from redacted import readSharppy

In [8]:
tamu_files = sorted(listdir('TAMU_TRACER_radiosonde_data_final/SHARPPY'))
tamu_dates = [dt.strptime(file.split('_')[2]+file.split('_')[3], '%Y%m%d%H%M') for file in tamu_files]
tamu_files = ['./TAMU_TRACER_radiosonde_data_final/SHARPPY/' + file for file in tamu_files]
dates_and_paths = {date.strftime('%Y-%m-%d %H:%M:%S'): path for date, path in zip(tamu_dates, tamu_files)}

In [9]:
panels = []
dataset_gbl = None

def plot_curve_theta(filename):
    dataset = update_dataset(filename)[0]
    pot_temp = mpcalc.potential_temperature(dataset.LEVEL, dataset.TEMP)
    pot_temp_curve = hv.Curve((pot_temp, dataset.HGHT), kdims=["Potential Temperature (K)"], vdims=["Height (m)"], label="Potential Temperature").opts(tools=['hover'], color='purple')
    return pot_temp_curve

def plot_curve_theta_deriv(filename, kernel_size):
    dataset = update_dataset(filename)[0]
    pot_temp = mpcalc.potential_temperature(dataset.LEVEL, dataset.TEMP)
    pot_temp_deriv = np.gradient(pot_temp.data.magnitude, dataset.HGHT.data.magnitude)
    pot_temp_deriv_avg = np.convolve(pot_temp_deriv, np.ones(kernel_size)/kernel_size, mode='same')
    pot_temp_deriv_curve = hv.Curve((pot_temp_deriv_avg, dataset.HGHT), kdims=["Potential Temperature Derivative (K/m)"], vdims=["Height (m)"], label="Potential Temperature Derivative").opts(tools=['hover'], color='purple')
    return pot_temp_deriv_curve

def plot_curve_wvmr(filename):
    dataset = update_dataset(filename)[0]
    wvmr = mpcalc.mixing_ratio_from_relative_humidity(dataset.LEVEL, dataset.TEMP, dataset.RH)
    wvmr_curve = hv.Curve((wvmr, dataset.HGHT), kdims=["Water Vapor Mixing Ratio (g/kg)"], vdims=["Height (m)"], label="Water Vapor Mixing Ratio").opts(tools=['hover'], color='green')
    return wvmr_curve

def plot_curve_wvmr_deriv(filename, kernel_size):
    dataset = update_dataset(filename)[0]
    wvmr = mpcalc.mixing_ratio_from_relative_humidity(dataset.LEVEL, dataset.TEMP, dataset.RH)
    wvmr_deriv = np.gradient(wvmr.data.magnitude, dataset.HGHT.data.magnitude)
    wvmr_deriv_avg = np.convolve(wvmr_deriv, np.ones(kernel_size)/kernel_size, mode='same')
    wvmr_deriv_curve = hv.Curve((wvmr_deriv_avg, dataset.HGHT), kdims=["Water Vapor Mixing Ratio Derivative (g/kg/m)"], vdims=["Height (m)"], label="Water Vapor Mixing Ratio Derivative").opts(tools=['hover'], color='green')
    return wvmr_deriv_curve

def plot_curve_thetav(filename):
    dataset = update_dataset(filename)[0]
    wvmr = mpcalc.mixing_ratio_from_relative_humidity(dataset.LEVEL, dataset.TEMP, dataset.RH)
    thetav = mpcalc.virtual_potential_temperature(dataset.LEVEL, dataset.TEMP, wvmr)
    thetav_curve = hv.Curve((thetav, dataset.HGHT), kdims=["Virtual Potential Temperature (K)"], vdims=["Height (m)"], label="Virtual Potential Temperature").opts(tools=['hover'], color='orange')
    return thetav_curve

def plot_curve_thetav_deriv(filename, kernel_size):
    dataset = update_dataset(filename)[0]
    wvmr = mpcalc.mixing_ratio_from_relative_humidity(dataset.LEVEL, dataset.TEMP, dataset.RH)
    thetav = mpcalc.virtual_potential_temperature(dataset.LEVEL, dataset.TEMP, wvmr)
    thetav_deriv = np.gradient(thetav.data.magnitude, dataset.HGHT.data.magnitude)
    thetav_deriv_avg = np.convolve(thetav_deriv, np.ones(kernel_size)/kernel_size, mode='same')
    thetav_deriv_curve = hv.Curve((thetav_deriv_avg, dataset.HGHT), kdims=["Virtual Potential Temperature Derivative (K/m)"], vdims=["Height (m)"], label="Virtual Potential Temperature Derivative").opts(tools=['hover'], color='orange')
    return thetav_deriv_curve

def plot_curve_wind_speed(filename):
    dataset = update_dataset(filename)[0]
    wind_speed_curve = hv.Curve((dataset.WSPD, dataset.HGHT), kdims=["Wind Speed (kt)"], vdims=["Height (m)"], label="Wind Speed").opts(tools=['hover'], color='blue')
    return wind_speed_curve

def plot_curve_wind_speed_deriv(filename, kernel_size):
    dataset = update_dataset(filename)[0]
    wind_speed_curve = hv.Curve((dataset.WSPD, dataset.HGHT), kdims=["Wind Speed (kt)"], vdims=["Height (m)"], label="Wind Speed").opts(tools=['hover'], color='blue')
    return wind_speed_curve

In [4]:
file_dropdown = pn.widgets.Select(name='Select a file', options=dates_and_paths, value=0)
kernel_size = pn.widgets.IntSlider(name='Kernel Size', start=1, end=10, value=5)

In [5]:
def update_dataset(filename):
    global dataset_gbl
    if dataset_gbl == None:
        print('READING (first time)!')
        print(filename)
        dataset_gbl, _, sounding_dt = readSharppy(filename)
        dataset_gbl.attrs['filename'] = filename
        dataset_gbl.attrs['datetime'] = sounding_dt
    elif dataset_gbl.attrs['filename'] != filename:
        print('READING (update)!')
        dataset_gbl, _, sounding_dt = readSharppy(filename)
        dataset_gbl.attrs['filename'] = filename
        dataset_gbl.attrs['datetime'] = sounding_dt
    else:
        sounding_dt = dataset_gbl.attrs['datetime']
    return dataset_gbl, sounding_dt
    

def plot_ABL_analysis(file_dropdown, kernel_size):
    print('replotting panes!')
    pot_temp_curve = hv.DynamicMap(pn.bind(plot_curve_theta, filename=file_dropdown, watch=True))

    pot_temp_pane = pn.pane.HoloViews(pot_temp_curve)
    
    wvmr_curve = hv.DynamicMap(pn.bind(plot_curve_wvmr, filename=file_dropdown, watch=True))
    wvmr_pane = pn.pane.HoloViews(wvmr_curve)
    
    thetav_curve = hv.DynamicMap(pn.bind(plot_curve_thetav, filename=file_dropdown, watch=True))
    thetav_pane = pn.pane.HoloViews(thetav_curve)
    
    wind_speed_curve = hv.DynamicMap(pn.bind(plot_curve_wind_speed, filename=file_dropdown, watch=True))
    wind_speed_pane = pn.pane.HoloViews(wind_speed_curve)

    thermo_panes = pn.Column(pn.Row(pot_temp_pane, wvmr_pane), pn.Row(thetav_pane, wind_speed_pane, watch=True))

    return thermo_panes

In [6]:
panels = pn.Column(file_dropdown, kernel_size, plot_ABL_analysis(file_dropdown, kernel_size))

replotting panes!


TypeError: Row.__init__() got an unexpected keyword argument 'watch'

In [ ]:
panels

READING (first time)!


/Users/stgardner4/Desktop/tracer-soundings/redacted.py:271: UserWarning: Vertical dimension number not found. Defaulting to initial dimension.
  soundingDS.attrs["k"] = mpcalc.k_index(soundingDS.LEVEL, soundingDS.TEMP, soundingDS.DWPT)
/Users/stgardner4/Desktop/tracer-soundings/redacted.py:272: UserWarning: Vertical dimension number not found. Defaulting to initial dimension.
  soundingDS.attrs["totaltotals"] = mpcalc.total_totals_index(soundingDS.LEVEL, soundingDS.TEMP, soundingDS.DWPT)
/Users/stgardner4/Desktop/tracer-soundings/redacted.py:273: UserWarning: Vertical dimension number not found. Defaulting to initial dimension.
  soundingDS.attrs["sweat"] = mpcalc.sweat_index(soundingDS.LEVEL, soundingDS.TEMP, soundingDS.DWPT, soundingDS.WSPD, soundingDS.WDIR)[0]


READING (first time)!


/Users/stgardner4/Desktop/tracer-soundings/redacted.py:271: UserWarning: Vertical dimension number not found. Defaulting to initial dimension.
  soundingDS.attrs["k"] = mpcalc.k_index(soundingDS.LEVEL, soundingDS.TEMP, soundingDS.DWPT)
/Users/stgardner4/Desktop/tracer-soundings/redacted.py:272: UserWarning: Vertical dimension number not found. Defaulting to initial dimension.
  soundingDS.attrs["totaltotals"] = mpcalc.total_totals_index(soundingDS.LEVEL, soundingDS.TEMP, soundingDS.DWPT)
/Users/stgardner4/Desktop/tracer-soundings/redacted.py:273: UserWarning: Vertical dimension number not found. Defaulting to initial dimension.
  soundingDS.attrs["sweat"] = mpcalc.sweat_index(soundingDS.LEVEL, soundingDS.TEMP, soundingDS.DWPT, soundingDS.WSPD, soundingDS.WDIR)[0]


READING (first time)!


/Users/stgardner4/Desktop/tracer-soundings/redacted.py:271: UserWarning: Vertical dimension number not found. Defaulting to initial dimension.
  soundingDS.attrs["k"] = mpcalc.k_index(soundingDS.LEVEL, soundingDS.TEMP, soundingDS.DWPT)
/Users/stgardner4/Desktop/tracer-soundings/redacted.py:272: UserWarning: Vertical dimension number not found. Defaulting to initial dimension.
  soundingDS.attrs["totaltotals"] = mpcalc.total_totals_index(soundingDS.LEVEL, soundingDS.TEMP, soundingDS.DWPT)
/Users/stgardner4/Desktop/tracer-soundings/redacted.py:273: UserWarning: Vertical dimension number not found. Defaulting to initial dimension.
  soundingDS.attrs["sweat"] = mpcalc.sweat_index(soundingDS.LEVEL, soundingDS.TEMP, soundingDS.DWPT, soundingDS.WSPD, soundingDS.WDIR)[0]


READING (first time)!


/Users/stgardner4/Desktop/tracer-soundings/redacted.py:271: UserWarning: Vertical dimension number not found. Defaulting to initial dimension.
  soundingDS.attrs["k"] = mpcalc.k_index(soundingDS.LEVEL, soundingDS.TEMP, soundingDS.DWPT)
/Users/stgardner4/Desktop/tracer-soundings/redacted.py:272: UserWarning: Vertical dimension number not found. Defaulting to initial dimension.
  soundingDS.attrs["totaltotals"] = mpcalc.total_totals_index(soundingDS.LEVEL, soundingDS.TEMP, soundingDS.DWPT)
/Users/stgardner4/Desktop/tracer-soundings/redacted.py:273: UserWarning: Vertical dimension number not found. Defaulting to initial dimension.
  soundingDS.attrs["sweat"] = mpcalc.sweat_index(soundingDS.LEVEL, soundingDS.TEMP, soundingDS.DWPT, soundingDS.WSPD, soundingDS.WDIR)[0]


READING (first time)! 
READING (first time)! 
READING (first time)! 
READING (first time)!

Column
    [0] Select(options={'2022-06-02 20:28:00': '....}, value='./TAMU_TRACER_radiosonde_...)
    [1] IntSlider(end=10, name='Kernel Size', start=1, value=5)
    [2] Column
        [0] Row
            [0] HoloViews(DynamicMap, height=300, sizing_mode='fixed', width=300)
            [1] HoloViews(DynamicMap, height=300, sizing_mode='fixed', width=300)
        [1] Row
            [0] HoloViews(DynamicMap, height=300, sizing_mode='fixed', width=300)
            [1] HoloViews(DynamicMap, height=300, sizing_mode='fixed', width=300)

In [11]:
hv.help(hv.Curve)

Curve

Online example: http://holoviews.org/reference/elements/bokeh/Curve.html

-------------
Style Options
-------------

	alpha, color, hover_alpha, hover_color, hover_line_alpha, hover_line_cap, hover_line_color, hover_line_dash, hover_line_join, hover_line_width, line_alpha, line_cap, line_color, line_dash, line_join, line_width, muted, muted_alpha, muted_color, muted_line_alpha, muted_line_cap, muted_line_color, muted_line_dash, muted_line_join, muted_line_width, nonselection_alpha, nonselection_color, nonselection_line_alpha, nonselection_line_cap, nonselection_line_color, nonselection_line_dash, nonselection_line_join, nonselection_line_width, selection_alpha, selection_color, selection_line_alpha, selection_line_cap, selection_line_color, selection_line_dash, selection_line_join, selection_line_width, visible

(Consult bokeh's documentation for more information.)

------------
Plot Options
------------

The plot options are the parameters of the plotting class:

Parameters of 